## An AI chatbot that specializes in Flight booking
## Demonstrates usage of Tools

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")


In [47]:
# Lets create a function to get seat availability given a destination city and date

seat_availability = {
                        "london":   [
                                        {
                                            "date":"01/01/2025",
                                            "seats" : 50
                                        },               
                                        {
                                            "date":"02/01/2025",
                                            "seats" : 20
                                        }
                                    ],
                        "paris":    [
                                        {
                                            "date":"01/01/2025",
                                            "seats" : 10
                                        },               
                                        {
                                            "date":"02/01/2025",
                                            "seats" : 1
                                        }
                                    ],
                        "tokyo":    [
                                        {
                                            "date":"01/01/2025",
                                            "seats" : 2
                                        },               
                                        {
                                            "date":"02/01/2025",
                                            "seats" : 10
                                        }
                                    ],



                    }

def get_seat_availability(destination_city, date):
    print(f"Tool get_seat_availability called for {destination_city} on {date}")
    city = destination_city.lower()
    # lets convert date from - to /
    date = date.replace("-", "/")
    
    if city in seat_availability:
        for availability in seat_availability[city]:
            print(f"Checking availability for {city} : {availability['date']} with {availability['seats']}")
            # check if the date is available
            
            if availability['date'] == date:
                print(f"Checking date for {city} : {availability['date']} with {availability['seats']}")
                return f"{availability['seats']} seats available"
    return "No Availability"
    

In [49]:
get_seat_availability("london", "01-01-2025")

Tool get_seat_availability called for london on 01-01-2025
Checking availability for london : 01/01/2025 with 50
Checking date for london : 01/01/2025 with 50


'50 seats available'

In [7]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [8]:
# Lets create a price function that will serve as a tools function with OpenAI


price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [9]:
# Lets add another tools function to get the flight seat availablity given a city and date
# This function will be called when the user asks for flight availability
availability_function = {
    "name": "get_flight_availability",
    "description": "Get the availability of flights to the destination city on a given date. Call this whenever you need to know the flight availability, for example when a customer asks 'Are there any flights to this city on this date?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "date": {
                "type": "string",
                "description": "The date of the flight in DD-MM-YYYY format",
            },
        },
        "required": ["destination_city", "date"],
        "additionalProperties": False
    }
}


In [15]:
tools = [
            {"type": "function", "function": price_function},
            {"type": "function", "function": availability_function}
        ]

In [11]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [12]:
# We have to write that function handle_tool_call:

def handle_tool_call_1(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    date = arguments.get('date')
    seats = get_seat_availability(city, date)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "date": date, "availability": seats}),
        "tool_call_id": tool_call.id
    }
    return response, city, date


In [50]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # if response.choices[0].finish_reason=="tool_calls":
    #    message = response.choices[0].message
    #    response, city = handle_tool_call(message)
    #    messages.append(message)
    #    messages.append(response)
    #    response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    # Lets handle the tool call
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        if message.tool_calls[0].function.name == "get_ticket_price":
            response, city = handle_tool_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
        elif message.tool_calls[0].function.name == "get_flight_availability":
            response, city, date = handle_tool_call_1(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content
    

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Tool get_seat_availability called for London on 01-01-2025
Checking availability for london : 01/01/2025 with 50
Checking date for london : 01/01/2025 with 50
Tool get_seat_availability called for London on 01-01-2025
Checking availability for london : 01/01/2025 with 50
Checking date for london : 01/01/2025 with 50
Tool get_seat_availability called for Tokyo on 02-01-2025
Checking availability for tokyo : 01/01/2025 with 2
Checking availability for tokyo : 02/01/2025 with 10
Checking date for tokyo : 02/01/2025 with 10
Tool get_ticket_price called for Tokyo
